# IMPORTS E CONFIGURAÇÕES

In [1]:
# Bibliotecas padrão para manipulação do sistema operacional
import os
import shutil
# Bibliotecas de web scrapping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
# Bibliotecas para manipulação de dados
import datetime
import time
import pandas as pd  # Manipulação de dados tabulares
# Biblioteca de avisos
import warnings


# Configurações
warnings.filterwarnings('ignore')
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
pd.options.display.float_format = lambda x: f'{x:,.2f}'

# WEB SCRAPING

## Download de uma arquivo anual

In [ ]:
# Download de arquivo excel no site
def download_one_excel_data(ano):
    # Iniciando o webdriver manager do navegador google chrome na sua ultima versão correspondente
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    if ano != 2022:
        # URL da página que contém o botão de download
        url = f"https://www.tesourotransparente.gov.br/publicacoes/transferencias-ao-fundo-de-manutencao-e-desenvolvimento-da-educacao-basica-fundeb/{ano}/114?ano_selecionado={ano}"
        # Acessando url
        driver.get(url)
        # Clicando no botão de download
        driver.find_element(
            by='xpath',
            value='//*[@id="publicacao"]/div/div[2]/section/div[4]/a').click()
        time.sleep(10)
        driver.quit()
    else:
        # URL da página que contém o botão de download
        url = 'https://www.tesourotransparente.gov.br/publicacoes/transferencias-ao-fundo-de-manutencao-e-desenvolvimento-da-educacao-basica-fundeb/2022/114-2?ano_selecionado=2022'
        # Acessando url
        driver.get(url)
        # Clicando no botão de download
        driver.find_element(
            by='xpath',
            value='//*[@id="publicacao"]/div/div[2]/section/div[4]/a').click()
        time.sleep(10)
        driver.quit()

## Encontrar arquivo na pasta destino

In [ ]:
# Nomeação e destinação de arquivo
def find_excel_file_name(ano):
    if ano in [2007, 2009, 2010, 2011, 2013]:
        origin = os.path.join(os.path.expanduser(
            '~'), 'Downloads', f'Fundeb{ano}.xls')
    elif ano == 2008:
        origin = os.path.join(os.path.expanduser(
            '~'), 'Downloads', f'Fundeb_{ano}.xls')
    elif ano in [2012, 2014, 2015, 2016, 2017]:
        origin = os.path.join(os.path.expanduser(
            '~'), 'Downloads', f'pge_fundeb_{ano}.xls')
    else:
        origin = os.path.join(os.path.expanduser(
            '~'), 'Downloads', f'Fundeb {ano}.xls')
    return origin

## Download de todos os arquivos anuais disponíveis

In [ ]:
# Download de todos os arquivos até o ano corrente (caso não hajam dentro da pasta)
def download_all_excel_datas(destiny_fold):
    # Criando variável do ano corrente
    corrent_year = datetime.datetime.today().year
    # Loop para baixar todos os arquivos a partir de 2007 até o ano corrente
    for ano in range(2007, corrent_year+1, 1):
        if f'Fundeb {ano}.xls' not in os.listdir(path=destiny_fold):
            print(f'Arquivo {ano} não encontrado, efetuando download.')
            # Efetuando o download do arquivo
            download_one_excel_data(ano)
            # Especificando pasta de destino padrão - Downloads
            origin = find_excel_file_name(ano=ano)
            # Especifique o caminho completo da pasta de destino
            destiny = f"{destiny_fold}/Fundeb {ano}.xls"
            # Ajustando pasta
            # Mova o arquivo para a pasta de destino
            shutil.move(origin, destiny)
        else:
            # Especifique o caminho completo do arquivo que você deseja verificar
            destiny = f"{destiny_fold}/Fundeb {ano}.xls"
            # Obtenha os metadados do arquivo, incluindo a data de modificação
            info = os.stat(destiny)
            # Acesse a data de modificação do arquivo
            last_modification = info.st_mtime
            # Converta a data de modificação para um formato legível
            data_formatada = time.ctime(last_modification)
            if (int(data_formatada[-5:]) < corrent_year) or (ano == corrent_year):
                print(f'Arquivo {ano} desatualizado, refazendo download.')
                # Efetuando o download do arquivo
                download_one_excel_data(ano)
                # Especificando pasta de destino padrão - Downloads
                origin = find_excel_file_name(ano=ano)
                # Substituindo o arquivo para a pasta de destino
                shutil.copy(origin, destiny)